In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [3]:
# Load ResNet-18 model
model = torchvision.models.resnet18(pretrained=False)

# Initialize layer-wise parameter counts
layer_params = {}

# Dynamically identify layers from model parameters
for name, _ in model.named_parameters():
    layer_name = name.split('.')[0]
    if layer_name not in layer_params:
        layer_params[layer_name] = 0

# Aggregate parameters by layer
for name, param in model.named_parameters():
    layer_name = name.split('.')[0]
    num_params = param.numel()
    layer_params[layer_name] += num_params
    
# Print results
print("ResNet-18 Layer-wise Parameter Count:")
print("=" * 60)

total_params = 0
for layer_name, params in layer_params.items():
    total_params += params
    print(f"{layer_name:50s} | {params:,}")

print("=" * 60)
print(f"{'Total Parameters':50s} | {total_params:,}")


ResNet-18 Layer-wise Parameter Count:
conv1                                              | 9,408
bn1                                                | 128
layer1                                             | 147,968
layer2                                             | 525,568
layer3                                             | 2,099,712
layer4                                             | 8,393,728
fc                                                 | 513,000
Total Parameters                                   | 11,689,512


In [4]:
# Initialize parameter groups
batchnorm_params = 0
bias_params = 0
other_params = 0

# Iterate through all named parameters
for name, param in model.named_parameters():
    if 'bn' in name and ('weight' in name or 'bias' in name):
        # Batch normalization affine parameters (weight and bias)
        batchnorm_params += param.numel()
    elif 'bias' in name:
        # Bias parameters from conv and fc layers
        bias_params += param.numel()
    else:
        # All other parameters (mainly weights from conv and fc layers)
        other_params += param.numel()

print("\nParameter Groups Breakdown:")
print("=" * 60)
print(f"{'Batch Normalization Parameters':50s} | {batchnorm_params:,}")
print(f"{'Bias Parameters (Conv & FC)':50s} | {bias_params:,}")
print(f"{'Other Parameters (mainly weights)':50s} | {other_params:,}")
print("=" * 60)
print(f"{'Total Parameters':50s} | {batchnorm_params + bias_params + other_params:,}")



Parameter Groups Breakdown:
Batch Normalization Parameters                     | 7,808
Bias Parameters (Conv & FC)                        | 1,896
Other Parameters (mainly weights)                  | 11,679,808
Total Parameters                                   | 11,689,512


In [ ]:
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

img = Image.open('spike.jpg').resize((200, 200))

# Apply each augmentation with specified parameters
augmentations = {}

# (a) ShearX - magnitude 0.3
augmentations['ShearX'] = TF.affine(
    img, angle=0, translate=[0, 0], scale=1.0, 
    shear=[np.degrees(np.arctan(0.3)), 0]
)

# (b) ShearY - magnitude 0.3
augmentations['ShearY'] = TF.affine(
    img, angle=0, translate=[0, 0], scale=1.0, 
    shear=[0, np.degrees(np.arctan(0.3))]
)

# (c) TranslateX - 40 pixels (20% of 200)
augmentations['TranslateX'] = TF.affine(
    img, angle=0, translate=[40, 0], scale=1.0, shear=[0, 0]
)

# (d) TranslateY - 40 pixels (20% of 200)
augmentations['TranslateY'] = TF.affine(
    img, angle=0, translate=[0, 40], scale=1.0, shear=[0, 0]
)

# (e) Rotate - 30 degrees
augmentations['Rotate'] = TF.rotate(img, angle=30)

# (f) Brightness - factor 1.8
augmentations['Brightness'] = TF.adjust_brightness(img, 1.8)

# (g) Color - saturation factor 1.8
augmentations['Color'] = TF.adjust_saturation(img, 1.8)

# (h) Contrast - factor 1.8
augmentations['Contrast'] = TF.adjust_contrast(img, 1.8)

# (i) Sharpness - factor 2.0
augmentations['Sharpness'] = TF.adjust_sharpness(img, 2.0)

# (j) Posterize - 4 bits
augmentations['Posterize'] = TF.posterize(img, 4)

# (k) Solarize - threshold 128
augmentations['Solarize'] = TF.solarize(img, 128)

# (l) Equalize - histogram equalization
augmentations['Equalize'] = TF.equalize(img)

# Create visualization
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle('RandAugment Transformations Demo', fontsize=20, fontweight='bold')

# Flatten axes for easier iteration
axes = axes.flatten()

# Plot original image
axes[0].imshow(img)
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Plot each augmentation
param_info = {
    'ShearX': 'magnitude=0.3',
    'ShearY': 'magnitude=0.3',
    'TranslateX': 'pixels=40',
    'TranslateY': 'pixels=40',
    'Rotate': 'degrees=30',
    'Brightness': 'factor=1.8',
    'Color': 'saturation=1.8',
    'Contrast': 'factor=1.8',
    'Sharpness': 'factor=2.0',
    'Posterize': 'bits=4',
    'Solarize': 'threshold=128',
    'Equalize': 'histogram eq.'
}

labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)', '(j)', '(k)', '(l)']

for idx, (name, aug_img) in enumerate(augmentations.items()):
    axes[idx + 1].imshow(aug_img)
    title = f'{labels[idx]} {name}\n{param_info[name]}'
    axes[idx + 1].set_title(title, fontsize=10, fontweight='bold')
    axes[idx + 1].axis('off')

# Hide remaining subplots
for idx in range(len(augmentations) + 1, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('randaugment_demo.png', dpi=300, bbox_inches='tight')
plt.show()

# Print parameter descriptions
print("\n" + "="*80)
print("PARAMETER DESCRIPTIONS")
print("="*80)
descriptions = {
    'ShearX': 'Applies shear transformation along X-axis (magnitude 0.3)',
    'ShearY': 'Applies shear transformation along Y-axis (magnitude 0.3)',
    'TranslateX': 'Shifts image horizontally by 40 pixels',
    'TranslateY': 'Shifts image vertically by 40 pixels',
    'Rotate': 'Rotates image by 30 degrees',
    'Brightness': 'Increases brightness by factor 1.8 (80% brighter)',
    'Color': 'Increases color saturation by factor 1.8 (80% more saturated)',
    'Contrast': 'Increases contrast by factor 1.8 (80% more contrast)',
    'Sharpness': 'Applies sharpness enhancement by factor 2.0',
    'Posterize': 'Reduces to 4 bits per channel (16 color levels)',
    'Solarize': 'Inverts all pixels below threshold 128',
    'Equalize': 'Applies histogram equalization for better contrast distribution'
}

for idx, (name, desc) in enumerate(descriptions.items()):
    print(f"{labels[idx]} {name:12s}: {desc}")

print("="*80)

NameError: name 'ImageEnhance' is not defined